In [ ]:
# Data colelction and Training 

In [ ]:
import cv2  # Importing OpenCV library for computer vision tasks
from cvzone.HandTrackingModule import HandDetector  # Importing HandDetector module from cvzone library
import numpy as np  # Importing NumPy library for numerical operations
import math  # Importing math library for mathematical operations
import time  # Importing time library for time-related functions

# Open the default camera (index 0)
capture = cv2.VideoCapture(0)

# Initializing HandDetector object with maximum hands to detect set to 2 for detecting both hands
handDetector = HandDetector(maxHands=2)

# Define an offset for cropping the hand region
cropOffset = 20

# Define the size of the output image
imageSize = 300

# Define the folder to save the captured images
savefolder = "Data/A"  # Define the folder to save the captured images

# Counter to keep track of the number of captured images
counter = 0

# Infinite loop to continuously capture frames from the camera
while True:
    # Read a frame from the camera
    suc, image = capture.read()

    # Detect hands in the frame
    hands, _ = handDetector.findHands(image)

    # If hands are detected
    if hands:
        # Loop through each detected hand
        for hand in hands:
            # Extract bounding box coordinates of the hand
            x, y, w, h = hand['bbox']

            # Create a white image with the specified size
            imageWhite = np.ones((imageSize, imageSize, 3), np.uint8) * 255

            # Crop the hand region from the frame
            imgageCrop = image[y - cropOffset:y + h + cropOffset, x - cropOffset:x + w + cropOffset]

            # Calculate the aspect ratio of the hand region
            aspRatio = h / w

            # If the aspect ratio is greater than 1, resize based on height
            if aspRatio > 1:
                k = imageSize / h
                widthCal = math.ceil(k * w)
                imgResize = cv2.resize(imgageCrop, (widthCal, imageSize))
                widthGap = math.ceil((imageSize - widthCal) / 2)
                imageWhite[:, widthGap:widthCal + widthGap] = imgResize
            # If the aspect ratio is less than or equal to 1, resize based on width
            else:
                k = imageSize / w
                heightCal = math.ceil(k * h)
                imageResize = cv2.resize(imgageCrop, (imageSize, heightCal))
                heightGap = math.ceil((imageSize - heightCal) / 2)
                imageWhite[heightGap:heightCal + heightGap, :] = imageResize

            # Display the resized hand image on a white background
            cv2.imshow("ImageWhite", imageWhite)

    # Display the original frame
    cv2.imshow("Image", image)

    # Wait for a key event and check if 's' is pressed
    key = cv2.waitKey(1)
    if key == ord("s"):
        # Increment the counter
        counter += 1
        # Save the cropped hand images with a timestamp as the filename
        for hand_index, hand in enumerate(hands):
            hand_image = hand['img']
            cv2.imwrite(f'{savefolder}/Hand_{hand_index}_Image_{time.time()}.jpg', hand_image)
        # Print the current count of saved images
        print(counter)


In [10]:
#TEST THE DATA


In [ ]:
import cv2  # Importing OpenCV library for computer vision tasks
from cvzone.HandTrackingModule import HandDetector  # Importing HandDetector module from cvzone library
from cvzone.ClassificationModule import Classifier  # Importing Classifier module from cvzone library
import numpy as np  # Importing NumPy library for numerical operations
import math  # Importing math library for mathematical operations

# Open the default camera (index 0)
capture = cv2.VideoCapture(0)

# Initializing HandDetector object with maximum hands to detect set to 2 for detecting both hands
detector = HandDetector(maxHands=2)

# Initializing Classifier object with the pre-trained model and labels
classifier = Classifier("Model/keras_model.h5", "Model/labels.txt")

# Define an offset for cropping the hand region
crop_offset = 20

# Define the size of the output image
image_size = 300

# Define the folder to save the captured images
save_folder = "Data/C"

# Counter to keep track of the number of captured images
counter = 0

# Labels for classification
labels = ["A", "B", "C", "D", "E", "F", "G", "H", "I", "J", "K", "L", "M", "N", "O", "P", "Q", "R", "S", "T", "U", "V", "W", "X", "Y", "Z"]

# Infinite loop to continuously capture frames from the camera
while True:
    # Read a frame from the camera
    success, image = capture.read()

    # Create a copy of the original frame for output visualization
    image_output = image.copy()

    # Detect hands in the frame
    hands, image = detector.findHands(image)

    # If hands are detected
    if hands:
        # Loop through each detected hand
        for hand in hands:
            # Extract bounding box coordinates of the hand
            x, y, w, h = hand['bbox']

            # Create a white image with the specified size
            image_white = np.ones((image_size, image_size, 3), np.uint8) * 255

            # Crop the hand region from the frame
            image_cropped = image[y - crop_offset:y + h + crop_offset, x - crop_offset:x + w + crop_offset]

            # Calculate the aspect ratio of the hand region
            aspect_ratio = h / w

            # If the aspect ratio is greater than 1, resize based on height
            if aspect_ratio > 1:
                k = image_size / h
                width_cal = math.ceil(k * w)
                image_resize = cv2.resize(image_cropped, (width_cal, image_size))
                width_gap = math.ceil((image_size - width_cal) / 2)
                image_white[:, width_gap:width_cal + width_gap] = image_resize
            # If the aspect ratio is less than or equal to 1, resize based on width
            else:
                k = image_size / w
                height_cal = math.ceil(k * h)
                image_resize = cv2.resize(image_cropped, (image_size, height_cal))
                height_gap = math.ceil((image_size - height_cal) / 2)
                image_white[height_gap:height_cal + height_gap, :] = image_resize

            # Get prediction and index from the classifier
            prediction, index = classifier.getPrediction(image_white, draw=False)

            # Draw rectangle and label on the output image
            cv2.rectangle(image_output, (x - crop_offset, y - crop_offset - 50), (x - crop_offset + 90, y - crop_offset - 50 + 50),
                          (255, 0, 255), cv2.FILLED)
            cv2.putText(image_output, labels[index], (x, y - 26), cv2.FONT_HERSHEY_COMPLEX, 1.7, (255, 255, 255), 2)
            cv2.rectangle(image_output, (x - crop_offset, y - crop_offset), (x + w + crop_offset, y + h + crop_offset), (255, 0, 255), 4)

    # Display the output frame
    cv2.imshow("Image", image_output)

    # Wait for a key event to exit the loop
    key = cv2.waitKey(1)
    if key == ord("q"):
        break  # Break the loop if 'q' key is pressed

    # Wait for a key event and check if 's' is pressed
    elif key == ord("s"):
        # Increment the counter
        counter += 1
        # Save the captured image with a timestamp as the filename
        cv2.imwrite(f'{save_folder}/Image_{counter}_{time.time()}.jpg', image_output)
        # Print the current count of saved images
        print(counter)

# Release the camera and close all OpenCV windows
capture.release()
cv2.destroyAllWindows()
